## Baseline modeling

In [5]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import cv2 as cv

from matplotlib import pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.utils import plot_model

In [4]:
tree_types = os.listdir('../data/image data/train')
X = []
y = []

for tree in tree_types:
    tree_path = os.path.join('../data/image data/train', tree)
    tree_disease_types = os.listdir(tree_path)

    for disease in tree_disease_types:
        img_name = os.listdir(os.path.join(tree_path, disease))

        for img in img_name:
            image_path = os.path.join(tree_path, disease, img)
            image = cv.resize(cv.imread(image_path), dsize=(256, 256))
            X.append(image)
            y.append([tree, disease])

In [7]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)
y

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [8]:
X

[array([[[196, 163, 167],
         [195, 162, 166],
         [194, 161, 165],
         ...,
         [211, 182, 185],
         [209, 180, 183],
         [207, 178, 181]],
 
        [[199, 166, 170],
         [198, 165, 169],
         [197, 164, 168],
         ...,
         [212, 183, 186],
         [210, 181, 184],
         [209, 180, 183]],
 
        [[199, 166, 170],
         [198, 165, 169],
         [197, 164, 168],
         ...,
         [214, 185, 188],
         [212, 183, 186],
         [211, 182, 185]],
 
        ...,
 
        [[124,  90, 100],
         [126,  92, 102],
         [128,  94, 104],
         ...,
         [137, 110, 126],
         [137, 110, 126],
         [137, 110, 126]],
 
        [[125,  91, 101],
         [126,  92, 102],
         [127,  93, 103],
         ...,
         [133, 106, 122],
         [134, 107, 123],
         [134, 107, 123]],
 
        [[128,  94, 104],
         [127,  93, 103],
         [126,  92, 102],
         ...,
         [132, 105, 121],
  

In [9]:
val_x = []
val_y = []

for tree in tree_types:
    tree_path = os.path.join('../data/image data/validation', tree)
    tree_disease_types = os.listdir(tree_path)

    for disease in tree_disease_types:
        img_name = os.listdir(os.path.join(tree_path, disease))

        for img in img_name:
            image_path = os.path.join(tree_path, disease, img)
            image = cv.resize(cv.imread(image_path), dsize=(256, 256))
            val_x.append(image)
            val_y.append([tree, disease])